In [8]:
import pandas as pd
import chromadb
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

In [3]:
price_data = pd.read_csv('BTC_1H.csv',sep='|')

price_data

,Time,Open,High,Low,Close,Volume,Taker Buy Quote,Taker Buy Base,Quote Asset Volume,Number of Trades
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,1.509525e+05,35.160503,2.023661e+05,171
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,9.260828e+04,21.448071,1.003048e+05,102
2,2017-08-17 06:00:00,4315.32,4345.45,4309.37,4324.35,7.229691,2.079532e+04,4.802861,3.128231e+04,36
3,2017-08-17 07:00:00,4324.35,4349.99,4287.41,4349.99,4.443249,1.129135e+04,2.602292,1.924106e+04,25
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,3.552747e+03,0.814655,4.239504e+03,28
...,...,...,...,...,...,...,...,...,...,...
58754,2024-04-30 06:00:00,63330.00,63465.10,63227.47,63412.00,482.606650,1.627014e+07,256.813450,3.057585e+07,29609
58755,2024-04-30 07:00:00,63412.00,63461.96,63135.70,63148.74,984.229000,3.142903e+07,496.415400,6.231255e+07,42758
58756,2024-04-30 08:00:00,63148.74,63244.00,61816.46,61919.32,4286.062850,1.170810e+08,1876.070850,2.675691e+08,140502
58757,2024-04-30 09:00:00,61919.32,62148.00,61600.00,61688.01,2706.323170,8.401635e+07,1357.327430,1.675057e+08,93692


In [5]:
llm_config = {
    "config_list": [
        {
            "model": "NotRequired",  # Loaded with LiteLLM command
            "api_key": "NotRequired",  # Not needed
            "base_url": "http://0.0.0.0:4000",  # Your LiteLLM URL
            "price": [0, 0],  # Put in price per 1K tokens [prompt, response] as free!
        }
    ],
    "cache_seed": 69,  # Turns off caching, useful for testing different models
}

In [7]:
analyst = RetrieveAssistantAgent(
    name='technicalAnalyst',
    system_message="You are an intelligent technical analyst that focuses on probability",
    llm_config=llm_config,
)

In [13]:
rag_analyst = RetrieveUserProxyAgent(
    name='ragTechnicalAnalyst',
    retrieve_config={
        'taks': 'qa',
        "docs_path": "BTC_1H.csv",
        'chunk_token_size': 2000,
        'vector_db': 'chroma',
    },
    code_execution_config={
        "use_docker": False
    }
)

/home/random/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
rag_analyst.retrieve_docs('BTC Price in 2024-01-01-13:00', n_results=10)

Trying to create collection.


max_tokens is too small to fit a single line of text. Breaking this line:
	Time|Open|High|Low|Close|Volume|Taker Buy Quote|Taker Buy Base|Quote Asset Volume|Number of Trades ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2024-07-31 17:29:38,852 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 1879 chunks.
2024-07-31 17:29:38,869 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.


VectorDB returns doc_ids:  [['9540eab6', '6311d47a', 'f48bc221', '36000a22', 'c7badea7', '4e3b3f54', 'ba26a608', '6e7cf0b4', 'f5bdb396', '3fb367d6']]


In [16]:


# Initialize the ChromaDB client
chromadb_client = chromadb.Client()

# Function to fetch document by ID
def fetch_document_by_id(doc_id):
    document = chromadb_client.get_document(doc_id)
    return document

# List of document IDs returned by the vector database
doc_ids = [['9540eab6', '6311d47a', 'f48bc221', '36000a22', 'c7badea7', '4e3b3f54', 'ba26a608', '6e7cf0b4', 'f5bdb396', '3fb367d6']]

# Flatten the list of document IDs
flat_doc_ids = [doc_id for sublist in doc_ids for doc_id in sublist]

# Fetch and print document contents
for doc_id in flat_doc_ids:
    document_content = fetch_document_by_id(doc_id)
    print(f"Document ID: {doc_id}")
    print(f"Content: {document_content}")
    print("\n")

AttributeError: 'Client' object has no attribute 'get_document'

In [17]:
query = 'What was the price of Bitcoin at 2024-01-01-13:00?'

chat_result = rag_analyst.initiate_chat(analyst, message=rag_analyst.message_generator, problem=query)
response = chat_result['response']
print(response)

VectorDB returns doc_ids:  [['05146701', '0d4b26b2', 'cb50a3fb', '671665e2', 'fc151bc5', 'b6021e87', '2d5f59ac', '32530f9e', 'c7badea7', '8194dc1a', 'f797942f', '729d1395', '2a49d1d9', '149c423c', '17cbd45c', '7f9bf139', '522063b1', '9eb259ba', 'a27e5db1', '3e4b408a']]
Adding content of doc 05146701 to context.
Adding content of doc 0d4b26b2 to context.
Adding content of doc cb50a3fb to context.
ragTechnicalAnalyst (to technicalAnalyst):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
If user's intent is code generation, you must obey the following rule

APIConnectionError: Connection error.